# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "weather_data.csv"
# output_data_file

types_df = pd.read_csv(output_data_file)

weather_data = types_df.drop(columns=["Unnamed: 0"])
weather_data


,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Narsaq,GL,1589910035,60.92,-46.05,55.40,19,20,11.41
1,Ushuaia,AR,1589910035,-54.80,-68.30,42.80,81,75,26.40
2,Rocha,UY,1589909945,-34.48,-54.33,72.61,63,3,8.63
3,Atuona,PF,1589910035,-9.80,-139.03,80.91,75,1,15.37
4,Rome,US,1589909883,43.21,-75.46,68.00,32,1,16.11
...,...,...,...,...,...,...,...,...,...
542,Urengoy,RU,1589910084,65.97,78.37,65.37,63,42,10.42
543,Astorga,BR,1589910084,-23.23,-51.67,82.40,39,20,11.41
544,Kyshtovka,RU,1589910084,56.55,76.63,62.33,80,66,7.27
545,Garoowe,SO,1589910084,8.41,48.48,92.57,36,60,14.76


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
gmaps.configure(api_key=g_key)

In [15]:
#locations
locations = weather_data[["Latitude", "Longitude"]]
#humidity
humidity = weather_data["Humidity"].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)
fig
print(fig)

Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 89 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 

new_types_df = pd.DataFrame(weather_data, columns = ["City", "Max Temperature", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temperature"] <= 89) & (new_types_df["Max Temperature"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new dataFrame
new_types_df[max_temp & wind_speed & cloudiness]

,City,Max Temperature,Wind Speed,Cloudiness
6,Castro,72.59,6.87,0
22,Kabinda,72.88,4.41,0
81,Daud Khel,85.68,3.13,0
93,Umluj,86.00,9.37,0
178,Auch,77.00,5.99,0
218,Mossamedes,78.91,6.49,0
296,Gaza,84.99,1.99,0
319,Chaman,73.26,1.21,0
336,Jiwani,86.47,5.41,0
380,Belen,86.00,8.05,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(weather_data, columns=["City", "Country", "Latitude", "Longitude"])

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Narsaq,GL,60.92,-46.05,
1,Ushuaia,AR,-54.80,-68.30,
2,Rocha,UY,-34.48,-54.33,
3,Atuona,PF,-9.80,-139.03,
4,Rome,US,43.21,-75.46,
...,...,...,...,...,...
542,Urengoy,RU,65.97,78.37,
543,Astorga,BR,-23.23,-51.67,
544,Kyshtovka,RU,56.55,76.63,
545,Garoowe,SO,8.41,48.48,


In [7]:

target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)
#---------

hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)
        
hotel_name


hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
#hotel_df.to_csv('hotel_export.csv')

KeyError: 'Lat'

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig